## Exploratory Data Analysis
We do some basic cleaning on the <i>sample.csv</i> input file of the <b>nyc_airbnb</b> project.

<b>Technical note:</b><br>
The library settings are given via project conditions. They are not up-to-date, older versions are used (like e.g. for the pandas profiling report).

### Imports

In [2]:
#
# imports
#

import wandb
import numpy as np
import pandas as pd
import pandas_profiling as pdp

### Start mlflow run with read-in sample csv input file
Dataframe creation afterwards...

In [3]:
# read-in of sample.csv file from w&b
run = wandb.init(project='nyc_airbnb', group='eda', save_code=True)
local_path = wandb.use_artifact('sample.csv:latest').file()
df_sample = pd.read_csv(local_path)

wandb: Currently logged in as: ilo (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [13]:
# first info of raw data set
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20000 non-null  int64  
 1   name                            19993 non-null  object 
 2   host_id                         20000 non-null  int64  
 3   host_name                       19992 non-null  object 
 4   neighbourhood_group             20000 non-null  object 
 5   neighbourhood                   20000 non-null  object 
 6   latitude                        20000 non-null  float64
 7   longitude                       20000 non-null  float64
 8   room_type                       20000 non-null  object 
 9   price                           20000 non-null  int64  
 10  minimum_nights                  20000 non-null  int64  
 11  number_of_reviews               20000 non-null  int64  
 12  last_review                     

### Creation of profiling report from raw data
Stored as .html file in the reports directory.

In [4]:
# create profiling report
profile = pdp.ProfileReport(df_sample, title="Pandas Profiling Report - Raw Sample DataFrame - NYC Airbnb")

In [5]:
profile.to_file("./reports/eda_sample_input_profiling_report.html")

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# to show the profile report in the notebook cell,
# does not happen with jupyter lab, therefore we have stored the report file
# profile.to_widgets()

### Some data cleaning
- outlier detection with 'price' feature: set min_price=10, max_price=350
- convert 'last_review' feature to type datetime
- skewed feature 'minimum_nights' shall get a normal distribution, because more reliable predictions are made if the predictors and the target variable are normally distributed: do simple log transformation

**Note:** We did not impute missing values. We will do that in the inference pipeline, so we will be able to handle missing values also in production.

In [6]:
# drop outliers
min_price = 10
max_price = 350
idx = df_sample['price'].between(min_price, max_price)
df_clean = df_sample[idx].copy()

In [7]:
# convert 'last_review' to datetime
df_clean['last_review'] = pd.to_datetime(df_clean['last_review'])

In [8]:
# normal distribution of 'minimum_nights'
df_clean['minimum_nights'] = np.log(df_clean['minimum_nights'])

In [9]:
df_clean.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,0.693147,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,1.098612,0,NaT,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,1.098612,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,1.098612,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,0.693147,8,2019-06-23,0.52,2,8


### Profiling of cleaned data

In [10]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  float64       
 11  nu

In [11]:
df_clean.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.900100e+04,1.900100e+04,19001.000000,19001.000000,19001.000000,19001.000000,19001.000000,15243.000000,19001.000000,19001.000000
mean,1.883041e+07,6.639459e+07,40.728063,-73.950827,122.340456,1.121220,23.797747,1.380928,6.583811,109.725383
std,1.096986e+07,7.782663e+07,0.055389,0.046825,71.530346,1.064895,45.493455,1.689988,31.154750,130.599899
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,10.000000,0.000000,0.000000,0.010000,1.000000,0.000000
25%,9.355498e+06,7.728754e+06,40.688820,-73.982050,66.000000,0.000000,1.000000,0.190000,1.000000,0.000000
50%,1.938754e+07,3.048785e+07,40.721710,-73.954630,100.000000,0.693147,6.000000,0.720000,1.000000,39.000000
75%,2.891952e+07,1.048354e+08,40.763210,-73.934490,160.000000,1.609438,24.000000,2.010000,2.000000,219.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,350.000000,7.130899,607.000000,27.950000,327.000000,365.000000


In [14]:
# create new profiling report of cleaned dataframe
profile = pdp.ProfileReport(df_clean, title="Pandas Profiling Report - Cleaned DataFrame - NYC Airbnb")

In [15]:
profile.to_file("./reports/eda_cleaned_df_profiling_report.html")

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Stop the mlflow run

In [13]:
run.finish()